In [1]:
# !pip install altair_saver
# !conda install -y -c conda-forge vega-cli vega-lite-cli
# !conda install -y -c conda-forge altair_saver

In [2]:
import json
from IPython.display import JSON

import pandas as pd
import altair as alt
# from altair_saver import save
import pydash as _
from altair import datum

In [38]:
# alt.renderers.enable('altair_saver')

RendererRegistry.enable('altair_saver')

In [4]:
labelTags = json.load(open('./tmp/labelTags.json'))
labelOptions = json.load(open('./tmp/labelOptions.json'))
allImages = json.load(open('../image_lists/all.json'))['images']

In [5]:
JSON(labelTags)

<IPython.core.display.JSON object>

In [6]:
JSON(labelOptions)

<IPython.core.display.JSON object>

In [7]:
# JSON(allImages)

In [8]:
# from pyppeteer import launch
# async def svg2png (in_svg_path, out_png_path):
#     browser = await launch(args=['--no-sandbox'])
#     page = await browser.newPage()
#     await page.goto(f"file://{os.path.abspath(in_svg_path)}")
#     dimensions = await page.evaluate('''() => {
#         return {
#             width: document.documentElement.width.baseVal.value,
#             height: document.documentElement.height.baseVal.value
#         }
#     }''')
#     await page.setViewport({'width': int(dimensions['width']), 'height': int(dimensions['height'])})
#     await page.screenshot({'path': out_png_path})
#     await browser.close()

## Statistics

In [9]:
issues = [{
    **_.pick(t, ['tag', 'name', 'count']),
    'namecount': f"{t['name']} ({t['count']})",
    'subcategory': _.find(labelOptions[0]['subcategories'], lambda x: x['subcategory'] == t['subcategories'][0])['name']
} for t in labelTags['tags'] if t['category'] == 'fault']

In [10]:
df_issues = pd.DataFrame(issues)

In [11]:
df_issues = pd.DataFrame(issues)
df_issues = df_issues.sort_values(['subcategory', 'count'], ascending=[True, False])
subcat_order = _.map_([o for o in labelOptions[0]['subcategories'] if o['group'] == 'pipeline'], 'name')
df_issues = df_issues.set_index('subcategory')
df_issues = df_issues.loc[subcat_order].reset_index(level=0)

In [12]:
issues_base = alt.Chart(df_issues).mark_bar().encode(
    x=alt.X('count:Q', title=None, scale=alt.Scale(domain=[0, 200])),
    y=alt.Y('namecount:N', sort='-x', title=None)
)

issues_data = issues_base.transform_filter(
    (datum.subcategory == 'Data')
).mark_bar(color='gray').properties(title='Data: Issues related to the underlying data')

issues_derive = issues_base.transform_filter(
    (datum.subcategory == 'Data Transformation')
).mark_bar(color='lightskyblue').properties(title='Data Transformation: Issues introduced when processing data')

issues_graph = issues_base.transform_filter(
    (datum.subcategory == 'Graph Drawing')
).mark_bar(color='ForestGreen').properties(title='Graph Drawing: Issues Introduced in the graph drawing process').encode(
    issues_base.encoding.x.copy()
)
issues_graph.encoding.x.title = 'Number of Visualizations (Non-exclusive)'

issues_visual = issues_base.transform_filter(
    (datum.subcategory == 'Reading')
).mark_bar(color='gold').properties(title='Reading: Difficulties in reading visualizations')

issues_perception = issues_base.transform_filter(
    (datum.subcategory == 'Perception')
).mark_bar(color='brown').properties(title='Perception: Issues related to human perception')

issues_logical = issues_base.transform_filter(
    (datum.subcategory == 'Message')
).mark_bar(color='purple').properties(title='Message: Visualizations that are trying to convey illogical message').encode(
    issues_base.encoding.x.copy()
)
issues_logical.encoding.x.title = 'Number of Visualizations (Non-exclusive)'

In [13]:
chart = (issues_data & issues_derive & issues_graph) | (issues_visual & issues_perception & issues_logical)
chart

TypeError: render() missing 1 required positional argument: 'fmts'

alt.HConcatChart(...)

In [39]:
save(chart, 'issues.svg')

/opt/conda/lib/vega-lite-cli/node_modules/vega-lite/build/vega-lite.js:4629
            throw new Error(message.INVALID_SPEC);
            ^

Error: Invalid spec
    at CoreNormalizer.mapLayerOrUnit (/opt/conda/lib/vega-lite-cli/node_modules/vega-lite/build/vega-lite.js:4629:19)
    at CoreNormalizer.map (/opt/conda/lib/vega-lite-cli/node_modules/vega-lite/build/vega-lite.js:4619:29)
    at CoreNormalizer.map (/opt/conda/lib/vega-lite-cli/node_modules/vega-lite/build/vega-lite.js:4939:26)
    at normalizeGenericSpec (/opt/conda/lib/vega-lite-cli/node_modules/vega-lite/build/vega-lite.js:5037:27)
    at normalize$1 (/opt/conda/lib/vega-lite-cli/node_modules/vega-lite/build/vega-lite.js:5030:16)
    at Object.compile (/opt/conda/lib/vega-lite-cli/node_modules/vega-lite/build/vega-lite.js:17051:26)
    at compile (/opt/conda/lib/vega-lite-cli/node_modules/vega-lite/bin/vl2vg:42:21)
    at /opt/conda/lib/vega-lite-cli/node_modules/vega-lite/bin/vl2vg:38:3
    at FSReqCallback.readFileAfter

CalledProcessError: Command '['/opt/conda/bin/vl2vg']' returned non-zero exit status 1.

In [15]:
datatypes = [{
    **_.pick(t, ['tag', 'name', 'count']),
    'namecount': f"{t['name']} ({t['count']})",
    'subcategory': _.find(labelOptions[2]['subcategories'], lambda x: x['subcategory'] == t['subcategory'])['name']
} for t in labelTags['tags'] if t['category'] == 'data']
df_datatypes = pd.DataFrame(datatypes)
df_datatypes = df_datatypes.sort_values(['subcategory', 'count'], ascending=[True, False])
subcat_order = _.map_(labelOptions[2]['subcategories'], 'name')
df_datatypes = df_datatypes.set_index('subcategory')
df_datatypes = df_datatypes.loc[subcat_order].reset_index(level=0)

In [16]:
alt.Chart(df_datatypes).mark_bar().encode(
    x=alt.X(field='count', type='quantitative', title='Number of Visualizations (Non-exclusive)', scale=alt.Scale(domain=[0, 750])),
    y=alt.Y(field='namecount', type='nominal', sort=list(df_datatypes['name']), title=None),
    color=alt.Color(field='subcategory', type='nominal', legend=alt.Legend(title=None), sort=subcat_order)
)

TypeError: render() missing 1 required positional argument: 'fmts'

alt.Chart(...)

In [17]:
charts = [{
    **_.pick(t, ['tag', 'name', 'count']),
    'namecount': f"{t['name']} ({t['count']})",
    'subcategory': _.find(labelOptions[1]['subcategories'], lambda x: x['subcategory'] == t['subcategory'])['name']
} for t in labelTags['tags'] if t['category'] == 'form']
df_charts = pd.DataFrame(charts)
df_charts = df_charts.sort_values(['subcategory', 'count'], ascending=[True, False])
subcat_order = _.map_(labelOptions[1]['subcategories'], 'name')
df_charts = df_charts.set_index('subcategory')
df_charts = df_charts.loc[subcat_order].reset_index(level=0)

In [18]:
alt.Chart(df_charts).mark_bar().encode(
    x=alt.X(field='count', type='quantitative', title='Number of Visualizations (Non-exclusive for multiple view visualizations)', scale=alt.Scale(domain=[0, 300])),
    y=alt.Y(field='namecount', type='nominal', sort=list(df_charts['name']), title=None),
    color=alt.Color(field='subcategory', type='nominal', legend=alt.Legend(title=None), sort=subcat_order)
)

TypeError: render() missing 1 required positional argument: 'fmts'

alt.Chart(...)

In [19]:
domains = [{
    **_.pick(t, ['tag', 'name', 'count']),
    'namecount': f"{t['name']} ({t['count']})" if t['name'] != 'Unknown' else f"*{t['name']} ({t['count']})",
} for t in labelTags['tags'] if t['category'] == 'domain']
df_domains = pd.DataFrame(domains)
df_domains = df_domains.sort_values(['count'], ascending=[False])
df_domains

,tag,name,count,namecount
11,domain:public,Public,197,Public (197)
5,domain:health,Health,139,Health (139)
0,domain:politics,Politics,134,Politics (134)
2,domain:entertainment,Entertainment,103,Entertainment (103)
1,domain:business,Business,100,Business (100)
8,domain:computer,Computer,94,Computer (94)
10,domain:international,International,84,International (84)
9,domain:environment,Environment,65,Environment (65)
12,domain:personal,Personal,56,Personal (56)
6,domain:education,Education,54,Education (54)


In [20]:
alt.Chart(df_domains).mark_bar(color='gray').encode(
    x=alt.X('count:Q', title='Number of Visualizations'),
    y=alt.Y('namecount:N', sort=list(df_charts['name']), title=None)
).properties(title='Data Domains')

TypeError: render() missing 1 required positional argument: 'fmts'

alt.Chart(...)

In [21]:
medium = [{
    **_.pick(t, ['tag', 'name', 'count']),
    'namecount': f"{t['name']} ({t['count']})",
} for t in labelTags['tags'] if t['category'] == 'media']
df_medium = pd.DataFrame(medium)
df_medium = df_medium.sort_values(['count'], ascending=[False])
df_medium

,tag,name,count,namecount
3,media:printed,Printed,95,Printed (95)
0,media:tv,TV,63,TV (63)
7,media:infographic,Infographic,23,Infographic (23)
2,media:ads,Ads,20,Ads (20)
1,media:news,News Article,15,News Article (15)
8,media:google,Google Products,15,Google Products (15)
4,media:handdrawn,Handdrawn,12,Handdrawn (12)
6,media:NSFW,NSFW,8,NSFW (8)
5,media:inreallife,Inreallife,5,Inreallife (5)


In [22]:
alt.Chart(df_medium).mark_bar(color='gray').encode(
    x=alt.X('count:Q', title='Number of Visualizations'),
    y=alt.Y('namecount:N', sort='-x', title=None)
).properties(title='Visualization Medium')

TypeError: render() missing 1 required positional argument: 'fmts'

alt.Chart(...)

In [23]:
effects = [{
    **_.pick(t, ['tag', 'name', 'count']),
    'namecount': f"{t['name']} ({t['count']})",
} for t in labelTags['tags'] if t['category'] == 'effect']
df_effects = pd.DataFrame(effects)
df_effects = df_effects.sort_values(['count'], ascending=[False])
df_effects

,tag,name,count,namecount
2,effect:confuesed,Confused,556,Confused (556)
1,effect:distorted,Distorted,477,Distorted (477)
3,effect:suggestive,Suggestive,25,Suggestive (25)
0,effect:reversed,Reversed,11,Reversed (11)


In [24]:
alt.Chart(df_effects).mark_bar(color='gray').encode(
    x=alt.X('count:Q', title='Number of Visualizations'),
    y=alt.Y('namecount:N', sort='-x', title=None)
).properties(title='Perceived Effects')

TypeError: render() missing 1 required positional argument: 'fmts'

alt.Chart(...)

## Co-occurence

### Charts and Issues

In [25]:
records = []
for issue in issues:
    for chart in charts:
        records.append({
            'Issue Types': issue['namecount'],
            'issue_tag': issue['tag'],
            'Chart Types': chart['namecount'],
            'chart_tag': chart['tag'],
            'count': len([image for image in allImages if issue['tag'] in image['labels'] and chart['tag'] in image['labels']])
        })

In [26]:
df_charts_issues = pd.DataFrame(records)

In [27]:
base = alt.Chart(df_charts_issues[df_charts_issues['count'] > 0]).encode(
    alt.X('Issue Types:N', scale=alt.Scale(paddingInner=0), sort=list(df_issues['namecount'])),
    alt.Y('Chart Types:N', scale=alt.Scale(paddingInner=0), sort=list(df_charts['namecount']))
)

# Configure heatmap
heatmap = base.mark_rect().encode(
    color=alt.Color('count:Q',
        scale=alt.Scale(scheme='blues', domain=[0, 100]),
        legend=alt.Legend(direction='vertical', title='# of Vis')
    )
)

# Configure text
text = base.mark_text(baseline='middle').encode(
    text='count:Q',
    color=alt.value('white')
)

# Draw the chart
heatmap + text

TypeError: render() missing 1 required positional argument: 'fmts'

alt.LayerChart(...)

### Data and Issues

In [28]:
records = []
for issue in issues:
    for datatype in datatypes:
        records.append({
            'Issue Types': issue['namecount'],
            'issue_tag': issue['tag'],
            'Data Types': datatype['namecount'],
            'data_tag': datatype['tag'],
            'count': len([image for image in allImages if issue['tag'] in image['labels'] and datatype['tag'] in image['labels']])
        })

In [29]:
df_datatypes_issues = pd.DataFrame(records)

In [30]:
base = alt.Chart(df_datatypes_issues[df_datatypes_issues['count'] > 0]).encode(
    alt.X('Issue Types:N', scale=alt.Scale(paddingInner=0), sort=list(df_issues['namecount'])),
    alt.Y('Data Types:N', scale=alt.Scale(paddingInner=0), sort=list(df_datatypes['namecount']))
)

# Configure heatmap
heatmap = base.mark_rect().encode(
    color=alt.Color('count:Q',
        scale=alt.Scale(scheme='blues', domain=[0, 100]),
        legend=alt.Legend(direction='vertical', title='# of Vis')
    )
)

# Configure text
text = base.mark_text(baseline='middle').encode(
    text='count:Q',
    color=alt.value('white')
)

# Draw the chart
heatmap + text

TypeError: render() missing 1 required positional argument: 'fmts'

alt.LayerChart(...)

### Domains and Issues

In [31]:
records = []
for issue in issues:
    for domain in domains:
        records.append({
            'Issue Types': issue['namecount'],
            'issue_tag': issue['tag'],
            'Domains': domain['namecount'],
            'domain_tag': domain['tag'],
            'count': len([image for image in allImages if issue['tag'] in image['labels'] and domain['tag'] in image['labels']])
        })

In [32]:
df_domains_issues = pd.DataFrame(records)

In [33]:
base = alt.Chart(df_domains_issues[df_domains_issues['count'] > 0]).encode(
    alt.X('Issue Types:N', scale=alt.Scale(paddingInner=0), sort=list(df_issues['namecount'])),
    alt.Y('Domains:N', scale=alt.Scale(paddingInner=0), sort=list(df_domains['namecount']))
)

# Configure heatmap
heatmap = base.mark_rect().encode(
    color=alt.Color('count:Q',
        scale=alt.Scale(scheme='blues', domain=[0, 100]),
        legend=alt.Legend(direction='vertical', title='# of Vis')
    )
)

# Configure text
text = base.mark_text(baseline='middle').encode(
    text='count:Q',
    color=alt.value('white')
)

# Draw the chart
heatmap + text

TypeError: render() missing 1 required positional argument: 'fmts'

alt.LayerChart(...)

### Domains and Charts

In [34]:
records = []
for chart in charts:
    for domain in domains:
        records.append({
            'Chart Types': chart['namecount'],
            'chart_tag': chart['tag'],
            'Domains': domain['namecount'],
            'domain_tag': domain['tag'],
            'count': len([image for image in allImages if chart['tag'] in image['labels'] and domain['tag'] in image['labels']])
        })
df_domains_charts = pd.DataFrame(records)
base = alt.Chart(df_domains_charts[df_domains_charts['count'] > 0]).encode(
    alt.X('Chart Types:N', scale=alt.Scale(paddingInner=0), sort=list(df_charts['namecount'])),
    alt.Y('Domains:N', scale=alt.Scale(paddingInner=0), sort=list(df_domains['namecount']))
)

# Configure heatmap
heatmap = base.mark_rect().encode(
    color=alt.Color('count:Q',
        scale=alt.Scale(scheme='blues', domain=[0, 100]),
        legend=alt.Legend(direction='vertical', title='# of Vis')
    )
)

# Configure text
text = base.mark_text(baseline='middle').encode(
    text='count:Q',
    color=alt.value('white')
)

# Draw the chart
heatmap + text

TypeError: render() missing 1 required positional argument: 'fmts'

alt.LayerChart(...)

### Effects and Issues

In [35]:
records = []
for issue in issues:
    for effect in effects:
        records.append({
            'Issue Types': issue['namecount'],
            'issue_tag': issue['tag'],
            'Effects': effect['namecount'],
            'effect_tag': effect['tag'],
            'count': len([image for image in allImages if issue['tag'] in image['labels'] and effect['tag'] in image['labels']])
        })
df_effects_issues = pd.DataFrame(records)
base = alt.Chart(df_effects_issues[df_effects_issues['count'] > 0]).encode(
    alt.X('Issue Types:N', scale=alt.Scale(paddingInner=0), sort=list(df_issues['namecount'])),
    alt.Y('Effects:N', scale=alt.Scale(paddingInner=0), sort=list(df_effects['namecount']))
)

# Configure heatmap
heatmap = base.mark_rect().encode(
    color=alt.Color('count:Q',
        scale=alt.Scale(scheme='blues', domain=[0, 100]),
        legend=alt.Legend(direction='vertical', title='# of Vis')
    )
)

# Configure text
text = base.mark_text(baseline='middle').encode(
    text='count:Q',
    color=alt.value('white')
)

# Draw the chart
heatmap + text

TypeError: render() missing 1 required positional argument: 'fmts'

alt.LayerChart(...)

### Effects and Charts

In [36]:
records = []
for chart in charts:
    for effect in effects:
        records.append({
            'Chart Types': chart['namecount'],
            'chart_tag': chart['tag'],
            'Effects': effect['namecount'],
            'effect_tag': effect['tag'],
            'count': len([image for image in allImages if chart['tag'] in image['labels'] and effect['tag'] in image['labels']])
        })
df_effects_charts = pd.DataFrame(records)
base = alt.Chart(df_effects_charts[df_effects_charts['count'] > 0]).encode(
    alt.X('Chart Types:N', scale=alt.Scale(paddingInner=0), sort=list(df_charts['namecount'])),
    alt.Y('Effects:N', scale=alt.Scale(paddingInner=0), sort=list(df_effects['namecount']))
)

# Configure heatmap
heatmap = base.mark_rect().encode(
    color=alt.Color('count:Q',
        scale=alt.Scale(scheme='blues', domain=[0, 100]),
        legend=alt.Legend(direction='vertical', title='# of Vis')
    )
)

# Configure text
text = base.mark_text(baseline='middle').encode(
    text='count:Q',
    color=alt.value('white')
)

# Draw the chart
heatmap + text

TypeError: render() missing 1 required positional argument: 'fmts'

alt.LayerChart(...)